## Nvidia Chip
## Jagadeesh Vasudevamurthy

# Write your code below
# You can use any number of private functions and classes

In [63]:
############################################################
#  class Exam
###########################################################    
class Exam():
    def __init__(self, show: 'bool'):
        self._show = show
        self._parent: dict[int, int] = {}
        self._size: dict[int, int] = {}
        self._graph: dict[int, set[int]] = {}

    def _log(self, *args, **kwargs):
        if self._show:
            print(*args, **kwargs)

    def _make_set(self, x: int) -> None:
        """Create a new isolated component if x is unseen."""
        if x not in self._parent:
            self._parent[x] = x
            self._size[x] = 1
            self._graph[x] = set()
            self._log(f"Created isolated component for {x}")

    def _find(self, x: int) -> int:
        """Find with path compression."""
        if self._parent[x] != x:
            self._parent[x] = self._find(self._parent[x])
        return self._parent[x]

    def _union(self, x: int, y: int) -> bool:
        """
        Returns True if two components were merged,
        False if they were already in the same group.
        """
        self._make_set(x)
        self._make_set(y)

        rx = self._find(x)
        ry = self._find(y)

        if rx == ry:
            self._log(f"Components {x} and {y} already connected (root {rx})")
            return False

        if self._size[rx] < self._size[ry]:
            rx, ry = ry, rx

        self._parent[ry] = rx
        self._size[rx] += self._size[ry]

        self._log(f"Union roots {rx} and {ry}; new size of root {rx} = {self._size[rx]}")
        return True

    ############################################################
    # Group component x with component y.
    # If x and y are already in the same group (directly or indirectly), return False.
    # Otherwise, put x and y in the same group and return True.
    ############################################################
    def group(self, x: int, y: int) -> bool:
        merged = self._union(x, y)
        if merged:
            self._graph[x].add(y)
            self._graph[y].add(x)
            self._log(f"Grouped components {x} and {y}")
        return merged

    ############################################################
    # Return the connection depth between component x and component y.
    # Return 1 if directly connected, 2 if connected through one intermediary, etc.
    # Lower values indicate more direct connectivity.
    ############################################################
    def connection_depth(self, x: int, y: int) -> int:
        # Unknown components: never part of any group
        if x not in self._parent or y not in self._parent:
            self._log(f"Connection depth requested for unknown components {x}, {y}")
            return -1

        # Not in the same connected component
        if self._find(x) != self._find(y):
            self._log(f"Components {x} and {y} are in different groups")
            return -1

        # Same component id
        if x == y:
            self._log(f"Connection depth requested for identical component {x}")
            return 0

        # Direct edge
        if y in self._graph[x]:
            self._log(f"Components {x} and {y} are directly connected")
            return 1

        # BFS to find minimal number of edges between x and y
        from collections import deque

        q = deque()
        visited = set()

        q.append((x, 0))
        visited.add(x)

        while q:
            node, dist = q.popleft()
            if node == y:
                self._log(f"Minimal connection depth between {x} and {y} is {dist}")
                return dist
            for nei in self._graph[node]:
                if nei not in visited:
                    visited.add(nei)
                    q.append((nei, dist + 1))

        self._log(f"Warning: no path found between {x} and {y} despite same DSU root")
        return -1

    ############################################################
    # Return the number of elements in the group containing component a.
    # For example, if a is in a group of size 5, returns 5.
    # If a has never been seen (not connected to any other component), returns 1
    ############################################################
    def group_size(self, a: int) -> int:
        if a not in self._parent:
            self._log(f"Group size for unseen component {a} = 1")
            return 1

        root = self._find(a)
        size = self._size[root]
        self._log(f"Group size for component {a} (root {root}) = {size}")
        return size

    ############################################################
    # Return a list where each item is the size of a connected group.
    # For example, [2, 2] means there are groups of sizes 2 and 1.
    ############################################################
    def component_sizes(self) -> list:
        roots = set()
        for node in self._parent:
            roots.add(self._find(node))

        sizes = [self._size[r] for r in roots]
        self._log(f"Component sizes: {sizes}")
        return sizes

    ############################################################
    # return number of unique components in the chip
    # Only counts elements that have appeared in group() calls.
    ############################################################
    def n(self) -> int:
        total = len(self._parent)
        self._log(f"Total number of unique components seen so far = {total}")
        return total


##  CANNOT CHANGE ANYTHING BELOW

## TEST BENCH
## NOTHING CAN BE CHANED BELOW

In [64]:
############################################################
# ExamTest.py 
# Test Bench for Exam
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2025
###########################################################

############################################################
#  NOTHING CAN BE CHANGED IN THIS FILE
########################################################### 

############################################################
#  All imports here
###########################################################
import sys # For getting Python Version
import random
#from Exam import *


############################################################
#  class  test factorial
###########################################################    
class Test_exam():
    def __init__(self):
        self._show = True 
        self._no = 0
        self._test_simple()
        print("You got 20 marks now")
        self._test_hidden()

    def assert_answer(self,a:'list', b:'list'):
        sa = sorted(a)
        sb = sorted(b)
        if (sa != sb):
            print("Your answer=",a)
            print("Expected answer=",b)
            assert(0)
            
    def _test_simple(self):
        self._test1()

    def _test1(self)->'void':
       e = Exam(True)
       n = e.group_size(1)
       assert(n == 1)
       
       n = e.group_size(5)
       assert(n == 1)
       
       x = e.group(1,2) 
       assert(x)

       x = e.group(1,2) 
       assert(x == False)

       x = e.group(2,1) 
       assert(x == False)

       n = e.group_size(2)
       assert(n == 2)

       n = e.group_size(1)
       assert(n == 2)

       l = e.connection_depth(1,2)
       assert(l < 5)
       assert(l == 1)

       x = e.group(3,4)
       assert(x)
       x = e.group(2,1) 
       assert(x == False)


       l = e.connection_depth(1,2)
       assert(l < 5)
       a = e.component_sizes()
       ans = [2,2]
       self.assert_answer(a,ans)

       x = e.group(2,4)
       assert(x)

       l = e.connection_depth(1,4)
       assert(l < 5)
       assert(l == 2)

       l = e.connection_depth(1,3)
       assert(l < 5)
       assert(l == 3)

       l = e.n()
       assert(l < 5)
       assert(l == 4)

       a = e.component_sizes()
       ans = [4]
       self.assert_answer(a,ans)

       n = e.group_size(2)
       assert(n == 4)

    def _test_hidden(self):
        print("I will run hidden tests after you submit")
        print("At this point you got only 20 marks")
 
############################################################
# MAIN
###########################################################    
def main():
    t = Test_exam()
    print("EXAM ENDS. Cannot post more than once in Canvas");
    print(sys.version)
    print(
"This material is copyrighted and strictly for registered students only.\n"
"Unauthorized copying, sharing, or posting in any electronic or physical form\n"
"is a violation of USA copyright law. Violators may face fines up to $250,000\n"
"per infringement and imprisonment of up to 5 years."
)


In [65]:
############################################################
# main
###########################################################
if (__name__    == '__main__'):
    main()

Group size for unseen component 1 = 1
Group size for unseen component 5 = 1
Created isolated component for 1
Created isolated component for 2
Union roots 1 and 2; new size of root 1 = 2
Grouped components 1 and 2
Components 1 and 2 already connected (root 1)
Components 2 and 1 already connected (root 1)
Group size for component 2 (root 1) = 2
Group size for component 1 (root 1) = 2
Components 1 and 2 are directly connected
Created isolated component for 3
Created isolated component for 4
Union roots 3 and 4; new size of root 3 = 2
Grouped components 3 and 4
Components 2 and 1 already connected (root 1)
Components 1 and 2 are directly connected
Component sizes: [2, 2]
Union roots 1 and 3; new size of root 1 = 4
Grouped components 2 and 4
Minimal connection depth between 1 and 4 is 2
Minimal connection depth between 1 and 3 is 3
Total number of unique components seen so far = 4
Component sizes: [4]
Group size for component 2 (root 1) = 4
You got 20 marks now
I will run hidden tests after